In [1]:
import pyximport; pyximport.install()
from gpcode import *
from csvparser import *

from functools import partial
import operator

In [2]:
#def readPopulationFromFile(heuristic_list_filename):
#    population = []
#    heuristic_file = open(heuristic_list_filename, 'r')
#    
#    for line in heuristic_file:
#        if len(line) > 3:
#            population.append(gp.PrimitiveTree.from_string(line, pset))
#    
#    heuristic_file.close()
#    
#    return population
csv_filename = "Comparison Results/mapping-" + machine_alias + ".csv.txt"

fields = [['Heuristic'], ['Complexity'], ['Fitness']]
conditions = [[operator.ge, 'Fitness', float, -0.25]]

csv_data = extractDictFromCSV(csv_filename)
parsed_data = extractIndividualsFromData(csv_data, ind_field=fields[0], complexity_field= fields[1], fitness_field=fields[2], condition=conditions)
parsed_data = sorted(parsed_data, reverse=True)

In [3]:
def createPopulationFromData(data):
    population = []
    for d in data:
        population.append(gp.PrimitiveTree.from_string(d.heuristic, pset))
    
    return population

In [4]:
def playMatchUp(p1_heuristic, p2_heuristic, n):
    p1 = HeuristicAI()
    p1.setHeuristic(marshal.dumps(toolbox.compile(p1_heuristic).func_code))
    p2 = HeuristicAI()
    p2.setHeuristic(marshal.dumps(toolbox.compile(p2_heuristic).func_code))

    return evaluateMatchUp(p1, p2, n)

In [5]:
def findFitnessMatchUp(population, opponent_heuristic, n):
    pop = population
    
    pool = mp.Pool(mp.cpu_count())
    
    p2 = HeuristicAI()
    p2.setHeuristic(marshal.dumps(toolbox.compile(opponent_heuristic).func_code))
    
    temp = []
    for i in range(0, len(pop)):
        temp.append(HeuristicAI())
        temp[i].setHeuristic(marshal.dumps(toolbox.compile(pop[i]).func_code))
    fitness_list = pool.map(partial(toolbox.evaluate, opponent=p2), temp)
    pool.terminate()
    
    for i in range(0, len(fitness_list)):
        pop[i].fitness = fitness_list[i]
    
    return pop

In [6]:
num_sims = 80000 * 10

#heuristic_list_filename = "Comparison Results/list_of_heuristics.txt"

total_population = createPopulationFromData(parsed_data)

csv_filename = "MatchUp Results/mapping-" + machine_alias + "-" + str(datetime.datetime.now()) + ".csv.txt"
if not os.path.isfile(csv_filename):
    createLogFile(csv_filename, "Num Sims; Player1; Fitness-p1; Player2; Fitness-p2;\n")

toolbox.register("evaluate", evaluateMatchUp, n=num_sims)
    
for i in range(len(total_population)-1):
    opponent = total_population[i]
    temp_pop = total_population[i+1:]
    temp_pop = findFitnessMatchUp(temp_pop, opponent, n=num_sims)
    for p in temp_pop:
        logToFile(csv_filename, str(num_sims) + "; " + str(p) + "; " + str(p.fitness[0]) + "; "  + str(opponent) + "; " + str(p.fitness[0] * -1) + ";\n")
